# Project 2

The goal of this assignment is for you to try out different ways of implementing and configuring a recommender, and to evaluate your different approaches.

For project 2, you’re asked to take some recommendation data (such as your toy movie dataset, Movielens, or another Dataset of your choosing), and implement at least two different recommendation algorithms on the data.  For example, content-based, user-user CF, and/or item-item CF.  You should evaluate different approaches, using different algorithms, normalization techniques, similarity methods, neighborhood sizes, etc.  You don’t need to be exhaustive—these are just some suggested possibilities.  You may use whatever third party libraries you want.  Please provide at least one graph, and a textual summary of your evaluation.

You may work in a small group.  Please submit a link to your GitHub repository for your Jupyter notebook or RMarkdown file.  Due end of day on Sunday June 26th.

**Requires the Jupyter-Scala language Kernel, available from: https://github.com/alexarchambault/jupyter-scala**

In [1]:
classpath.add( "org.apache.spark" %% "spark-core" % "1.6.1",
             "org.apache.spark" %% "spark-mllib" % "1.6.1",
              "org.apache.spark" %% "spark-sql" % "1.6.1")

158 new artifact(s)


158 new artifacts in macro
158 new artifacts in runtime
158 new artifacts in compile


# Response

## The Recommender System

This week I'll try loading a more complicated dataset: Plain text. For the purpose of this exercise, I'll generate a Recipe-Recipe similarity model that generates recommendations based on the ingredients of the recipes. 

I'll focus on Appetizer Recipes from http://mc6help.tripod.com/RecipeLibrary/RecipeLibrary.htm

## The Code

### Firing up a Spark Context

In [2]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.distributed.{MatrixEntry, RowMatrix}

import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.distributed.{MatrixEntry, RowMatrix}

In [3]:

    val conf = new SparkConf()
      .setAppName("week1-EstimatePi")
      .setMaster("local") 

    val sc = new SparkContext(conf)


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
16/06/25 23:37:42 INFO SparkContext: Running Spark version 1.6.1
16/06/25 23:37:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/06/25 23:37:43 INFO SecurityManager: Changing view acls to: malarconba001
16/06/25 23:37:43 INFO SecurityManager: Changing modify acls to: malarconba001
16/06/25 23:37:43 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(malarconba001); users with modify permissions: Set(malarconba001)
16/06/25 23:37:44 INFO Utils: Successfully started service 'sparkDriver' on port 45083.
16/06/25 23:37:44 INFO Slf4jLogger: Slf4jLogger started
16/06/25 23:37:44 INFO Remoting: Starting remoting
16/06/25 23:37:45 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkDriverActorSystem@192.168.1.15:45096]
16/06/25 23:37:45 INFO Utils: Su

conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@29058a95
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@3950693d

### Data Loading and Transformations

The objective here is to:

* Load the http://mc6help.tripod.com/RecipeLibrary/AllAppetizerRecipes.txt file
* Transform into Zero filled matrix
* Transform into Long-format data structure


In [4]:

val csv = 
    sc
        .textFile("AllAppetizerRecipes.txt")
        .map(t => t.trim)
        .map(t => (t,t=="* Exported from MasterCook *") ) // add boolean if we have a record delimiter
        .zipWithIndex // add record id
        .map(r=>(r._1._1,r._1._2,r._2)) // flatten the nested index
csv.take(20)



csv: org.apache.spark.rdd.RDD[(String, Boolean, Long)] = MapPartitionsRDD[5] at map at Main.scala:30
res3_1: Array[(String, Boolean, Long)] = Array(
  ("* Exported from MasterCook *", true, 0L),
  ("", false, 1L),
  ("Barbecue Pecans", false, 2L),
  ("", false, 3L),
  ("Recipe By     : Possum Kingdom Lake Cookbook", false, 4L),
  ("Serving Size  : 25    Preparation Time : 0:00", false, 5L),
  ("Categories    :", false, 6L),
  ("Amount  Measure       Ingredient -- Preparation Method", false, 7L),
  ("--------  ------------  --------------------------------", false, 8L),
  ("2   tablespoons  butter", false, 9L),
  ("1/4           cup  Worcestershire sauce", false, 10L),
  ("1    tablespoon  catsup", false, 11L),
  ("6        dashes  Hot sauce", false, 12L),
  ("4          cups  Pecans -- halves", false, 13L),
  ("salt -- to taste", false, 14L),
  ("", false, 15L),
  (
    "Melt butter in a large saucepan; add Worcestershire sauce, , catsup, and hot sauce.",
...

From the above, sample, we can see that the text file gets imported a line at a time per record. However, the recipes file is formatted as follows:

```
* Exported from MasterCook *

                     Barbecue Pecans

Recipe By     : Possum Kingdom Lake Cookbook
Serving Size  : 25    Preparation Time : 0:00
Categories    : 
  Amount  Measure       Ingredient -- Preparation Method
--------  ------------  --------------------------------
       2   tablespoons  butter
     1/4           cup  Worcestershire sauce
       1    tablespoon  catsup
       6        dashes  Hot sauce
       4          cups  Pecans -- halves
                        salt -- to taste

Melt butter in a large saucepan; add Worcestershire sauce, , catsup, and hot sauce.  

Stir in nuts; spoon into a glass baking dish, spreading evenly.  toast at 400 degrees about 20 minutes, stirring frequently.  

Turn out on absorbent towels, and sprinkle with salt.

                                    - - - - - - - - - - - - - - - - - - - 
```

The goal here is to map the file into: RecipeIngredients(RecipeName, Ingredient, IsUsed) 

Where:

* Record Break: is the line: ```* Exported from MasterCook *```
* An entire record is composed of the lines between record breaks
* RecipeName: The third line in the record
* Ingredient: The third column in the Ingredients table
* IsUsed: Defaults to 1.

In [5]:
// The goal with this section is to end up with a recipeIndexTable collection that looks like: recipeIndexTable(RecordId,RecipeID)

// Generate a recordIndex table (RecipeId, LineId) by relying in the presence and offset of the record break
val recordIndexes =  csv.filter(_._2).map(_._3).zipWithIndex.map(r=>(r._2,r._1)) ++ sc.parallelize(Seq((csv.filter(_._2).count,csv.count)))

// Now, let's just create an offset recordIndexes Table 
val recordIndexesOffset = recordIndexes.map(r=>(r._1-1,r._2-1))

// and join it with the record indexes so we have the recipeIndexTable(RecordId,RecipeID)
val recipeIndexTable = recordIndexes.join(recordIndexesOffset).flatMap(r=> (r._2._1 to r._2._2).map(t=>(t,r._1)))
recipeIndexTable.collect.sorted

recordIndexes: org.apache.spark.rdd.RDD[(Long, Long)] = UnionRDD[12] at $plus$plus at Main.scala:27
recordIndexesOffset: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[13] at map at Main.scala:30
recipeIndexTable: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[17] at flatMap at Main.scala:33
res4_3: Array[(Long, Long)] = Array(
  (0L, 0L),
  (1L, 0L),
  (2L, 0L),
  (3L, 0L),
  (4L, 0L),
  (5L, 0L),
  (6L, 0L),
  (7L, 0L),
  (8L, 0L),
  (9L, 0L),
  (10L, 0L),
  (11L, 0L),
  (12L, 0L),
  (13L, 0L),
  (14L, 0L),
  (15L, 0L),
  (16L, 0L),
  (17L, 0L),
  (18L, 0L),
...

Let's now join it to the imported data so we add the recipeID

In [6]:


val csvIndexed = csv.map(r=>(r._3,(r._1,r._2))).join(recipeIndexTable)
csvIndexed.take(4)

csvIndexed: org.apache.spark.rdd.RDD[(Long, ((String, Boolean), Long))] = MapPartitionsRDD[21] at join at Main.scala:27
res5_1: Array[(Long, ((String, Boolean), Long))] = Array(
  (
    3558L,
    (
      (
        "Blend well the cream cheese with the Brie cheese. Add the hazelnuts and apple; blend. Spread on melba toast or crackers.",
        false
      ),
      92L
    )
  ),
  (1084L, (("- - - - - - - - - - - - - - - - - - -", false), 28L)),
  (3586L, (("2            tb  Olive oil", false), 93L)),
  (1410L, (("", false), 37L))
)

Let's now combine the recipe lines separated by a pipe (|) 

In [7]:
val recipesText = csvIndexed
    .map(r=>(r._2._2,r._1,r._2._1._1))// let's flatten the nested list so we have RecipeId, Recipe Line Id and Recipe Line
    .sortBy(r=>(r._1,r._2)) // Properly sort it so we have all lines consecutively arranged as per the recipe line id
    .map(r=>(r._1,r._3)) // retain only the recipeid and recipe lines
    .groupBy(_._1) // and group it by the RecipeId
    .map( 
        g => (
                g._1,    // return the RecipeId
                g._2.map(_._2.trim).mkString("|")  // and concatenate the nested array of recipe lines with a pipe
        )
    )

recipesText.take(4)

recipesText: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[31] at map at Main.scala:30
res6_1: Array[(Long, String)] = Array(
  (
    34L,
    "* Exported from MasterCook *||Cheese-Olive Balls||Recipe By     :|Serving Size  : 1     Preparation Time : 0:00|Categories    :|Amount  Measure       Ingredient -- Preparation Method|--------  ------------  --------------------------------|1/4      teaspoon  hot pepper sauce|1      teaspoon  paprika|1/2      teaspoon  salt|2          cups  sharp cheddar cheese -- grated|1/2           cup  butter|1           cup  flour -- sifted|olives||Mix ingredientsexcept olives  like a pie crust.  Wrap each olive with mixture.  spread the little balls on a pan and freeze.  Bake at 425 degrees for 12 minutes. Can be keep frozen in a bag. Serve hot.||||||||||- - - - - - - - - - - - - - - - - - -|||||Nutr. Assoc. : 0"
  ),
  (
    52L,
    "* Exported from MasterCook *||Stuffed Ripe Olives||Recipe By     : The Army Time Magazine/Nov. 13, 1978|Serv

And finally, let's extract our data in a long format: RecipeIngredients(RecipeName,RecipeNameHash, IngredientName,IngredientNameHash, IsUsed) 

In [8]:
import scala.util.matching.Regex
var recipeIngredients = recipesText
    .flatMap{
        text=>
            val recipeName = {        // get the recipe name from the first part of the recipe record
                val pattern = new Regex("""\* Exported from MasterCook \*\|\|([^\|]+)\|\|""", "RecipeName") 
                pattern.findFirstMatchIn(text._2).get.group("RecipeName")  
            }

            """(?s)----\|.*?\|\|""".r   /* Let's find the recipe table, which is all the stuff 
                                                 between the header and 2 consecutive pipes (new lines) */
                .findFirstMatchIn(text._2)    // get the first match
                .map(r=>r.matched)            // and return the matched text from the regex object
                .getOrElse("")                // get or default to whitespace
                .split("\\|")                 // and now process the table lines
                .map(
                    _.trim.replaceAll(" {2,}","\\|")  // for each line, let's pipe delimit the columns.
                )
                .filter(r=>(!r.isEmpty)&&(r!="----")&&(r.length>2)) // filtering out the empty ones and short aberrations
                .map { r=>   // for each recipe line
                    val v = r.split("\\|")    // split it into an iterator by the pipe sign
                    (recipeName, v(v.length-1),1.00) // amd return (RecipeName, IngredientName, IsUSed)
                }
    }.distinct

recipeIngredients.collect

import scala.util.matching.Regex
recipeIngredients: org.apache.spark.rdd.RDD[(String, String, Double)] = MapPartitionsRDD[35] at distinct at Main.scala:47
res7_2: Array[(String, String, Double)] = Array(
  ("Fresh Gazpacho", "Green Pepper -- chopped", 1.0),
  (
    "Cucumber Rye Surprises, Pk",
    """
cucumbers -- sliced 1/8" thick
    """,
    1.0
  ),
  ("Cheese-Olive Balls", "flour -- sifted", 1.0),
  ("Fruit Kabobs", "grapes", 1.0),
  ("Bacon Rollups", "bacon", 1.0),
  ("Cheese Straws", "butter", 1.0),
  ("Rhode Island Clam Cakes", "pepper", 1.0),
  ("Baked Whole Garlic with French Bread", "black pepper", 1.0),
  ("Dip For Sausage Balls", "worcestershire sauce", 1.0),
  ("Crunchy Chocolate-Coconut Balls", "whole wheat flakes -- crisp", 1.0),
  (
    "Buffalo-Style Crab Claws",
    "Alaska snow crab or blue -- crab cocktail claws",
...

Lets have a freestanding list of recipes and ingredients

In [9]:
val ingredients = recipeIngredients.map(_._2).distinct.zipWithIndex
ingredients.collect
val recipes = recipeIngredients.map(_._1).distinct.zipWithIndex
recipes.collect

ingredients: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[40] at zipWithIndex at Main.scala:25
res8_1: Array[(String, Long)] = Array(
  ("bread", 0L),
  ("French bread loaf -- sliced and heated", 1L),
  ("cucumber -- thinly sliced", 2L),
  ("pine nuts -- toasted", 3L),
  ("soy sauce", 4L),
  ("-- crumbled", 5L),
  ("sour cream -- or yogurt", 6L),
  ("Salt -- pepper", 7L),
  ("Hot sauce", 8L),
  ("salt, to taste", 9L),
  ("chicken pieces -- see notes", 10L),
  ("hot chili oil", 11L),
  ("lettuce leaves", 12L),
  ("Mushrooms -- coarsely chopped", 13L),
  ("catsup", 14L),
  ("chopped pecans -- or walnuts", 15L),
  ("bean sprouts", 16L),
  ("minced parsley", 17L),
  ("segmented", 18L),
...
recipes: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[45] at zipWithIndex at Main.scala:31
res8_3: Array[(String, Long)] = Array(
  ("Crunchy Chocolate-Coconut Balls", 0L),
  ("Shirlie's Cheese Ball", 1L),
  ("Corn 'n Bacon Sticks, Pk", 2L),
  ("Rice Krispies Balls, Pk",

Let's finally bake the indexes along with the recipeIngredients

In [10]:
val withIngredientsIndexed = recipeIngredients.groupBy(_._2).join(ingredients)
withIngredientsIndexed.take(4)

withIngredientsIndexed: org.apache.spark.rdd.RDD[(String, (Iterable[(String, String, Double)], Long))] = MapPartitionsRDD[50] at join at Main.scala:27
res9_1: Array[(String, (Iterable[(String, String, Double)], Long))] = Array(
  ("bread", (CompactBuffer(("Chicken Almond Dainties", "bread", 1.0)), 0L)),
  (
    "French bread loaf -- sliced and heated",
    (
      CompactBuffer(
        (
          "Baked Whole Garlic with French Bread",
          "French bread loaf -- sliced and heated",
          1.0
        )
      ),
      1L
    )
  ),
  (
    "cucumber -- thinly sliced",
    (CompactBuffer(("Snack Sandwiches", "cucumber -- thinly sliced", 1.0)), 2L)
  ),
  (
...

It's time to flatten the group

In [11]:
val withIngredientsIndexedFlat = withIngredientsIndexed.flatMap(r => r._2._1.map(x => (x._1,x._2,r._2._2,x._3)))
withIngredientsIndexedFlat.take(4)

withIngredientsIndexedFlat: org.apache.spark.rdd.RDD[(String, String, Long, Double)] = MapPartitionsRDD[51] at flatMap at Main.scala:25
res10_1: Array[(String, String, Long, Double)] = Array(
  ("Chicken Almond Dainties", "bread", 0L, 1.0),
  (
    "Baked Whole Garlic with French Bread",
    "French bread loaf -- sliced and heated",
    1L,
    1.0
  ),
  ("Snack Sandwiches", "cucumber -- thinly sliced", 2L, 1.0),
  (
    "Mushrooms Filled with Feta Cheese and Pine Nuts",
    "pine nuts -- toasted",
    3L,
    1.0
  )
)

And add the recipe ID following the same methodology:

In [12]:
val recipeIngredientsIndexed = withIngredientsIndexedFlat
    .groupBy(_._1)
    .join(recipes)
    .flatMap(r => r._2._1.map(x => (x._1,r._2._2,x._2,x._3,x._4)))

recipeIngredientsIndexed.take(4)

recipeIngredientsIndexed: org.apache.spark.rdd.RDD[(String, Long, String, Long, Double)] = MapPartitionsRDD[57] at flatMap at Main.scala:30
res11_1: Array[(String, Long, String, Long, Double)] = Array(
  ("Crunchy Chocolate-Coconut Balls", 0L, "egg -- slightly beaten", 148L, 1.0),
  ("Crunchy Chocolate-Coconut Balls", 0L, "sweet chocolate squares", 169L, 1.0),
  ("Crunchy Chocolate-Coconut Balls", 0L, "butter", 285L, 1.0),
  ("Crunchy Chocolate-Coconut Balls", 0L, "coconut flakes", 322L, 1.0)
)

### Converting the RDD into a Sql Context

Thinking that it may be useful, here it is:


In [13]:
// lot's of code from : http://spark.apache.org/docs/latest/sql-programming-guide.html

// sc is an existing SparkContext.
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@7770f984

In [14]:
// Import Row.
import org.apache.spark.sql.Row;

// Import Spark SQL data types
import org.apache.spark.sql.types.{StructType,StructField,StringType,DoubleType};
// Generate the schema based on the string of schema
val schema =
  StructType(List(StructField("recipeName", StringType, true)
             ,StructField("ingredientName", StringType, true)
             ,StructField("isUsed", DoubleType, true)
            )
            )

// Convert records of the RDD (people) to Rows.
val rowRDD = recipeIngredients.map(p => Row(p._1, p._2,p._3))

// Apply the schema to the RDD.
val recipeIngredientsDataFrame = sqlContext.createDataFrame(rowRDD, schema)

// Register the DataFrames as a table.
recipeIngredientsDataFrame.registerTempTable("recipeingredients")



import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructType,StructField,StringType,DoubleType}
schema: org.apache.spark.sql.types.StructType = StructType(
  StructField(recipeName,StringType,true),
  StructField(ingredientName,StringType,true),
  StructField(isUsed,DoubleType,true)
)
rowRDD: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[58] at map at Main.scala:41
recipeIngredientsDataFrame: org.apache.spark.sql.DataFrame = [recipeName: string, ingredientName: string, isUsed: double]

Let's query the dataframe

In [15]:
sqlContext.sql("SELECT * FROM recipeingredients where upper(ingredientName) like '%CILANTRO%'").collect

res14: Array[org.apache.spark.sql.Row] = Array(
  [Fruit Salsa Dip,cilantro,1.0],
  [Tuna Appetizers,dried cilantro,1.0],
  [Southwestern Chicken Filo Triangles,fresh cilantro -- minced,1.0],
  [Southwestern Chicken Filo Triangles,fresh cilantro -- finely minced,1.0],
  [Pot Stickers,Cilantro -- minced,1.0],
  [Crab And Avocado Cocktail,cilantro; fresh -- snipped,1.0],
  [Vietnamese Spring Rolls,cilantro -- minced,1.0]
)

I thought I would use the SQL collection to pivot the matix. However, the performance is so poor when I ran it that I'm completely ommiting it from the excercise.

### Transforming into a Matrix and Generating the Cosine Similarities

In [16]:
// Helper function that displays a nicely formatted table
def displayTable(table:List[Map[String, String]])(implicit publish: jupyter.api.Publish[jupyter.api.Evidence]): Unit = {
    val keys = table.flatMap(r=>r.keys).distinct.sorted
    val header = "<th>"+keys.mkString("</th><th>")+"</th>"
    val rows = "<tr>"+table.map(r=>keys.map(k=>"<td>"+r.getOrElse(k,"&nbsp;")+"</td>")).mkString("</tr><tr>")+"</tr>"
    publish.display("table",("text/html" -> ("<table>"+header+rows+"</table>")))
}

defined function displayTable

In [17]:
// I don't yet know why but I have to get these counts and hard code them into the functor. 
ingredients.count.toInt
recipes.count.toInt

res16_0: Int = 626
res16_1: Int = 103

In [18]:
// it also sucks that the only silimarity function is buried in a RowMatrix object, which only takes dense vectors
val recipeIngredientsMatrix = new RowMatrix(
    recipeIngredientsIndexed
        .groupBy(_._2)
        .map(
            r=>
                Vectors.sparse(628,r._2.map(i=> (i._4.toInt,i._5)).toSeq)
        )
)

recipeIngredientsMatrix.rows.take(5)

val cs = recipeIngredientsMatrix.columnSimilarities

val ic = ingredients.collect
cs.entries
  .map {
    case MatrixEntry(i, j, u) => (i, j, u) }
  .collect
//  .map(r => (ic.filter(ri => ri._2 == r._1).map(_._1), ic.filter(ri => ri._2 == r._2).map(_._1), r._3.toDouble))
  .map(r => (ic.filter(ri => ri._2 == r._1).map(_._1), ic.filter(ri => ri._2 == r._2).map(_._1), r._3.toDouble))
  .sortBy(-_._3)

recipeIngredientsMatrix: org.apache.spark.mllib.linalg.distributed.RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@13b669c3
res17_1: Array[org.apache.spark.mllib.linalg.Vector] = Array(
  (628,[147,199,286,439,501],[1.0,1.0,1.0,1.0,1.0]),
  (628,[99,124,135,228,285,382,472,487],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]),
  (628,[115,282,583],[1.0,1.0,1.0]),
  (628,[10,99,165,184,200,302,397],[1.0,1.0,1.0,1.0,1.0,1.0,1.0]),
  (628,[75,250,326,389,415,446,531,532,572],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])
)
cs: org.apache.spark.mllib.linalg.distributed.CoordinateMatrix = org.apache.spark.mllib.linalg.distributed.CoordinateMatrix@10ece095
ic: Array[(String, Long)] = Array(
  ("bread", 0L),
  ("French bread loaf -- sliced and heated", 1L),
  ("cucumber -- thinly sliced", 2L),
  ("pine nuts -- toasted", 3L),
  ("soy sauce", 4L),
  ("-- crumbled", 5L),
  ("sour cream -- or yogurt", 6L),
  ("Salt -- pepper", 7L),
  ("Hot sauce", 8L),
  ("salt, to taste", 9L),
  ("chicken pieces -- se

In [19]:
val ingredientsRecipeMatrix = new RowMatrix(
    recipeIngredientsIndexed
        .groupBy(_._4)
        .map(
            r=>
                Vectors.sparse(103,r._2.map(i=> (i._2.toInt,i._5)).toSeq)            
        )
)

ingredientsRecipeMatrix.rows.take(5)

val cs = ingredientsRecipeMatrix.columnSimilarities

// let's collect the ingredients so we can see them
val recipeIngredientsLocal = recipeIngredients.collect

val rc = recipes.collect

// format the recipe-recipe as an html table for display purposes
val cse = cs.entries
      .map {
        case MatrixEntry(i, j, u) => (i, j, u) }
      .collect
      .map(r => (rc.filter(ri => ri._2 == r._1).map(_._1), rc.filter(ri => ri._2 == r._2).map(_._1), r._3.toDouble))
      .sortBy(-_._3)
      .map(r=> Map(" Recipe 1"->r._1.mkString
                   ,"Ingredients 1"->recipeIngredientsLocal.filter(_._1==r._1.mkString).map(_._2).mkString("<br>")
                   ," Recipe 2"->r._2.mkString
                   ,"Ingredients 2"->recipeIngredientsLocal.filter(_._1==r._2.mkString).map(_._2).mkString("<br>")
                   ,"  Cosine Similarity"->r._3.toString))


ingredientsRecipeMatrix: org.apache.spark.mllib.linalg.distributed.RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@1554fd67
res18_1: Array[org.apache.spark.mllib.linalg.Vector] = Array(
  (103,[74],[1.0]),
  (103,[3],[1.0]),
  (103,[41],[1.0]),
  (103,[69],[1.0]),
  (103,[91],[1.0])
)
cs: org.apache.spark.mllib.linalg.distributed.CoordinateMatrix = org.apache.spark.mllib.linalg.distributed.CoordinateMatrix@5e6fe153
recipeIngredientsLocal: Array[(String, String, Double)] = Array(
  ("Fresh Gazpacho", "Green Pepper -- chopped", 1.0),
  (
    "Cucumber Rye Surprises, Pk",
    """
cucumbers -- sliced 1/8" thick
    """,
    1.0
  ),
  ("Cheese-Olive Balls", "flour -- sifted", 1.0),
  ("Fruit Kabobs", "grapes", 1.0),
  ("Bacon Rollups", "bacon", 1.0),
  ("Cheese Straws", "butter", 1.0),
  ("Rhode Island Clam Cakes", "pepper", 1.0),
  ("Baked Whole Garlic with French Bread", "black pepper", 1.0),
  ("Dip For Sausage Balls", "worcestershire sauce", 1.0),
  ("Crunchy Chocolate-

## The Recipe-Recipe Simliarity Based Model

Let's see the top-20 similar reciples

In [20]:
displayTable(
    cse.toList.take(20)
    )

0.47809144373375745,Cranberry Coconut Fruit Balls,Orange Coconut Balls,dried apricotsgraham cracker crumbsfresh cranberries -- rinsed and drainedred food coloringconfectioner's sugarcoconut flakespecansgrated orange peel -- from 1 orangegreen food coloringbutter,watergraham cracker crumbschopped pecansconfectioner's sugarbuttercoconut flakesfrozen orange juice concentrate -- thawed
0.4629100498862757,Crunchy Chocolate-Coconut Balls,Orange Coconut Balls,whole wheat flakes -- crispsweet chocolate squaresconfectioner's sugarcoconut flakesegg -- slightly beatenbutter,watergraham cracker crumbschopped pecansconfectioner's sugarbuttercoconut flakesfrozen orange juice concentrate -- thawed
0.44721359549995787,"Toasted ID Bits, Pk",Debbie's Spiced Pecans,shelled peanutsworcestershire saucebutter -- meltedbite-size shredded wheattabasco saucegarlic powdercelery saltbite-size oat cereal ringspretzel sticks,worcestershire saucebutter -- meltedtabasco saucepecan halvesgarlic salt
0.4082482904638631,Curried Pecans,"Curried Meat Balls, Pk",pecan halves -- or walnutsmelted buttercurry powdersalt,minced steakdry bread crumbscurry powderegg -- beatensaltfew grains pepper
0.3872983346207417,Crunchy Chocolate-Coconut Balls,Cranberry Coconut Fruit Balls,whole wheat flakes -- crispsweet chocolate squaresconfectioner's sugarcoconut flakesegg -- slightly beatenbutter,dried apricotsgraham cracker crumbsfresh cranberries -- rinsed and drainedred food coloringconfectioner's sugarcoconut flakespecansgrated orange peel -- from 1 orangegreen food coloringbutter
0.3749999999999999,Baked Whole Garlic with French Bread,Rosemary Chicken Wings,black pepperclarified buttergarlic -- left wholesaltbutter -- softenedFrench bread loaf -- sliced and heatedcream cheese or soft cheese (optional)olive oil,saltolive oilbutterfinely chopped shallotsblack pepperchicken wingsdried rosemarylemonade
0.3380617018914066,Sardine Appetizer,Copenhagens,mayonnaisechopped olivestoasted breadlemon juiceminced pimentostuffed olivesmashed sardines,"chopped parsleymayonnaiselemon juicewaterchestnuts -- canned, drainedshrimp"
0.3333333333333334,"Corn 'n Bacon Sticks, Pk","Pork Meatball With Sweet-Sour Sauce, Pk",-- crumbledmargarine or butter -- meltedcream-style corn -- cannedBisquick baking mixgarlic saltbacon -- crisply fried and,ground porkBisquick baking mixegggarlic powdersweet-sour sauce -- see recipegarlic salt
0.32732683535398865,"Curried Meat Balls, Pk",Antipasti,minced steakdry bread crumbscurry powderegg -- beatensaltfew grains pepper,"smoked mozzarella cheese -- cut into 3/4 cubessaltjulienned fresh basil leavesItalian seasoningjulienned sun-dried tomatoes in olive oil -- drainedbalsamic vinegar -- optionalripe olivesmixed greens, such as mesclun, gourmet, Itroasted red peppers -- drainedegg -- beatendry bread crumbsdried basil -- optionalolive oilartichoke heart quarters -- drained"
0.31622776601683794,Cocktail Beer Ball,Sweet and Sour Party Meat Balls,vinegarsugarground chuckbeercatsupworcestershire sauce,worcestershire saucevinegarhot sauceground roundground porkpineapple chunks in syrup -- drainedsugarfinely chopped onionpeppercornstarchsoft bread crumbssalteggs -- beatengreen pepper -- dicedtomato sauce
0.3162277660168379,Sugar Walnuts,Ella's Divine Date Rum Balls,honeywatersaltwalnutssugar,dates -- pitted and choppedsugarrice krispies.butterconfectioner's sugar -- optionalchopped pecansrumsalt


## Plotting it:

Scala/Spark does not offer much plotting options. For convenience, let's embeed a static plotly graph. I'll eventually figure out how to dynamically pass data to the graph from my app

In [21]:
publish.display("table",("text/html" -> ("""<iframe width="900" height="800" frameborder="0" scrolling="no" src="https://plot.ly/~rmalarc/3.embed"></iframe>""")))

In [22]:
sc.stop

# Conclusions

* Spark is raw power, not bells and whistles and user friendliness: Here are some of the major limitations/issues I've found
  * Lot's of datatypes disparate APIs. For instance, the cosine similarity is buried into a Matrix api, which you can't directly use if you have a plain RDD or SQL dataframe. Lot's of data conversions
  * I only found that ONE similarity function (which is the cosine). I didn't get to experiment with other functions.
  * I couldn't get it to reference other RDDs within another's functor. For instanace, I had to hardcode the length of the recipe's array as I couldn't directly access it within the second iterator. Due to the same reason, I had to first calculate whatever I wanted to access from within the functor, and bake it into the RDD by using joins. Perhaps this is the Spark way. Lot's of "Task not serializable" erros: http://stackoverflow.com/questions/22592811/task-not-serializable-java-io-notserializableexception-when-calling-function-ou
* It was really cool to be able to completely parse a plain text list of recipes into a "dataset".
* The cosine simmilarity seems to work pretty nicely, even in a really sparse scenario such as this one. Better results should be obtained by mastering the list of ingredients.